In [6]:
import cv2
from deepface import DeepFace

# Cargar las imágenes de superposición
lipstick_image = cv2.imread('../P5/images/labios2.png', cv2.IMREAD_UNCHANGED)
mustache_image = cv2.imread('../P5/images/mostacho.png', cv2.IMREAD_UNCHANGED)

# Verificar si las imágenes se han cargado correctamente
if lipstick_image is None:
    print("Error: No se pudo cargar la imagen de labios en 'path/to/lipstick.png'.")
if mustache_image is None:
    print("Error: No se pudo cargar la imagen de bigote en 'path/to/mustache.png'.")

# Convertir las imágenes a BGRA si no tienen canal alfa
if lipstick_image is not None and lipstick_image.shape[2] == 3:
    lipstick_image = cv2.cvtColor(lipstick_image, cv2.COLOR_BGR2BGRA)
if mustache_image is not None and mustache_image.shape[2] == 3:
    mustache_image = cv2.cvtColor(mustache_image, cv2.COLOR_BGR2BGRA)

def overlay_image(background, overlay, position):
    x, y = position
    h, w = overlay.shape[:2]

    # Superponer donde no hay transparencia en overlay
    alpha_overlay = overlay[:, :, 3] / 255.0  # Canal alfa de overlay
    alpha_background = 1.0 - alpha_overlay

    for c in range(0, 3):  # Para cada canal de color
        background[y:y+h, x:x+w, c] = (alpha_overlay * overlay[:, :, c] +
                                       alpha_background * background[y:y+h, x:x+w, c])
    return background

# Iniciar la cámara
vid = cv2.VideoCapture(0)

while True:
    ret, frame = vid.read()
    if not ret:
        break

    try:
        # Análisis del rostro
        result = DeepFace.analyze(frame, actions=['gender'], enforce_detection=False)

        # Obtener el género dominante
        gender = result[0]['dominant_gender']  # Verificamos que está en result[0]

        # Detección de la región facial
        faces = DeepFace.extract_faces(frame, detector_backend="retinaface", align=False)
        if len(faces) > 0 and 'facial_area' in faces[0]:
            facial_area = faces[0]['facial_area']
            x, y, w, h = facial_area['x'], facial_area['y'], facial_area['w'], facial_area['h']
            
            # Aplicar superposición basada en el género
            if gender == 'Man' and lipstick_image is not None:
                overlay_img = cv2.resize(lipstick_image, (w, h // 2))
                frame = overlay_image(frame, overlay_img, (x, y + h // 2))
            elif gender == 'Woman' and mustache_image is not None:
                overlay_img = cv2.resize(mustache_image, (w, h // 4))
                frame = overlay_image(frame, overlay_img, (x, y + h // 2))

        # Muestra el frame con la superposición
        cv2.imshow("Filtro de Género", frame)
    
    except Exception as e:
        print("Error:", e)

    # Esc para salir
    if cv2.waitKey(1) & 0xFF == 27:
        break

# Cerrar cámara y ventanas
vid.release()
cv2.destroyAllWindows()


Error: Face could not be detected in numpy array.Please confirm that the picture is a face photo or consider to set enforce_detection param to False.
